In [63]:
# 1

import lib
import importlib
import math
import random
importlib.reload(lib)

simx = lambda : random.normalvariate(0, 1)
stats = lib.RecurrentStats()

while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > 0.1:
    stats.add(simx())



print(stats.n)
print(stats.get_mean)
print(stats.get_var)
print(math.sqrt(stats.get_var))




101
-0.0905375891032869
0.7552437826694243
0.8690476296897796


In [98]:
# 2

import lib
import importlib
import math
import random
importlib.reload(lib)

# a)
print("a")
# i

"""
Como esta integral es la esperanza E[(exp(x) / sqrt(2 * x))] con x ~ U(0,1)
Podemos hacerla usando Montecarlo directamente
"""
gi = lambda x : (math.exp(x) / math.sqrt(2 * x))
x = lib.montecarlo_multi(gi, [(0, 1)], 10_000)
print(f"i) {x}")

# ii
"""
A esta la separamos en dos: (-inf, 0) + (0, inf)
Haciendo un cambio de variable sobre la parte negativa, nos quedan dos integrales:

integral 0 a inf de f(x) dx + integral 0 a inf de f(-x) dx

en el apunte vimos como aproximar funciones de 0 a infinito. Usamos esa técnica abajo
"""
gii = lambda x : x**2 * math.exp(-x**2)
y = lib.montecarlo_multi(gii, [(0, math.inf)], 100_000) + lib.montecarlo_multi(lambda x : gii(-x), [(0, math.inf)], 100_000)


print(f"ii) {y}")

print("b")

# uso el metodo de montecarlo con n = 1, lo que me da una sola sample en vez de un promedio
# abajo hago el promedio parando cuando llegamos a la cota
simi = lambda : lib.montecarlo_multi(gi, [(0, 1)], 1)
simii = lambda : lib.montecarlo_multi(gii, [(0, math.inf)], 1) + lib.montecarlo_multi(lambda x : gii(-x), [(0, math.inf)], 1)

stats = lib.RecurrentStats()
while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > 0.01:
    stats.add(simi())
print("i) ", stats.mean)

stats = lib.RecurrentStats()
while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > 0.01:
    stats.add(simii())
print("ii)", stats.mean)



a
i) 2.0795610174226904
ii) 0.8887385388118403
b
i)  2.073377895501431
ii) 0.893259762690186


In [108]:
# 3

import lib
import importlib
import math
import random
importlib.reload(lib)


gi  = lambda x : math.sin(x) / x

simi  = lambda : lib.montecarlo_multi(gi, [(math.pi, 2*math.pi)], 1)
stats = lib.RecurrentStats()
zalpha_2 = 1.96 # alpha = 0.05
L = 2 * 0.001
d = L / (2 * zalpha_2)
while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > d:
    stats.add(simi())
print(stats.get_mean)
print(stats.n)

gii = lambda x : 3 / (3 + x**4)
simii = lambda : lib.montecarlo_multi(gii, [(0, math.inf)], 1)
stats = lib.RecurrentStats()
while (stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > d):
    stats.add(simii())
print(stats.get_mean)
print(stats.n)

-0.43436454959897297
169649
1.4622045389862766
3659704


In [6]:
import lib
import importlib
import math
import random
importlib.reload(lib)

# 4

print("b")

def simN():
    n = 0
    s = 0
    while s <= 1:
        s += random.random()
        n += 1
    return n

# esp, var = lib.sim_esp_var(simN, 10_000)
# print(esp, var)


print("c")
stats = lib.RecurrentStats()
zalpha_2 = 1.96 # alpha = 0.05
L = 0.025
d = L / (2 * zalpha_2)
while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > d:
    stats.add(simN())
print(stats.get_mean)

b
c
2.726426928705825


In [1]:
import lib
import importlib
import math
import random
importlib.reload(lib)

# 5

def simM():
    n = 1
    u_1 = random.random()
    
    while True:
        u_2 = random.random()
        n += 1


        if u_1 > u_2:
            break
        u_1 = u_2
        u_2 = random.random()
    return n


print("c")
stats = lib.RecurrentStats()
d = 0.01
while stats.n <= 100 or (stats.get_var/stats.n) > d:
    stats.add(simM())
print(stats.get_mean)
print(stats.n)

print("d")
stats = lib.RecurrentStats()
zalpha_2 = 1.96 # alpha = 0.05
L = 0.1
d = L / (2 * zalpha_2)
while stats.n <= 100 or math.sqrt(stats.get_var/stats.n) > d:
    stats.add(simM())
print(stats.get_mean)
print(stats.n)

c
2.7425742574257432
101
d
2.7576419213973806
1374


In [143]:

import lib
import importlib
import math
import random
importlib.reload(lib)
# 6

def g():
    x = 2 * random.random() - 1
    y = 2 * random.random() - 1
    return x**2 + y**2 <= 1

stats = lib.RecurrentStats()

while stats.n <= 100 or (stats.get_var/stats.n) > 0.0001:
    stats.add(g())
print("a")
print(stats.get_mean)
print(stats.n)

print("b")
L = 0.1
zalpha_2 = 1.96
d = L/(2 * zalpha_2)
stats = lib.RecurrentStats()

while stats.n <= 100 or math.sqrt(stats.get_var/stats.n) > d:
    stats.add(g())
print(stats.get_mean)
print(stats.n)

a
0.8113577023498683
1532
b
0.7302631578947371
304


In [67]:
import numpy as np
from scipy.stats import chi2
import lib
import importlib
import math
import random
importlib.reload(lib)
# 7 

# Distribucion teorica de Mendel 
# blanco, rojo, rosa
# 0, 1, 2
ps = [1/4, 1/4, 1/2]

print("a")

# Calculamos el estadistico T
# e 141 produjeron flores blancas, 291 flores rosas y 132 flores rojas
Ns = [141, 132, 291]
T = lib.estadistico_prueba(ps, Ns)


print("Valor de T: ", T)
print("n: ", sum(Ns))
print("p-valor: ", chi2.sf(T, 2))


print("b")
pvalsim = 0
m = 1000
for _ in range(m):
    pvalsim += int(lib.estadistico_prueba_sim(ps, sum(Ns)) >= T) / m

print("p-valor Simulado: ", pvalsim)





a
Valor de T:  0.8617021276595745
n:  564
p-valor:  0.6499557054800363
b
p-valor Simulado:  0.6570000000000005


In [69]:
import numpy as np
from scipy.stats import chi2
import lib
import importlib
import math
import random
importlib.reload(lib)

#(i = 1,2,3,4,5,6) fue, respectivamente, 158, 172, 164, 181, 160, 165
ps = [1/6] * 6
Ns = 158, 172, 164, 181, 160, 165

T = lib.estadistico_prueba(ps, Ns)

pval = chi2.sf(T, len(ps) - 1)

print("T ", T)
print("p-valor ", pval)

n = 1000
pvalsim = 0
for _ in range(n):
    pvalsim += int(lib.estadistico_prueba_sim(ps, sum(Ns)) >= T) / n
print("p-valor simulado", pvalsim)


T  2.18
p-valor  0.8237195392577814
p-valor simulado 0.8230000000000006


In [30]:
import numpy as np
from scipy.stats import chi2
import lib
import importlib
import math
import random
importlib.reload(lib)

# 9

sample = [0.12, 0.18, 0.06, 0.33, 0.72, 0.83, 0.36, 0.27, 0.77, 0.74]
d = lib.estadistico_smirnov(sample, lambda x : x)
pvalor = 0

m = 10_000

for _ in range(m):
    pvalor += int(lib.estadistico_smirnov_sim(len(sample)) > d) / m

print("d: ", d)
print("p-valor: ", pvalor)

d:  0.24
p-valor:  0.5333999999999576


In [51]:
import numpy as np
from scipy.stats import chi2, expon
import lib
import importlib
import math
import random
importlib.reload(lib)
# 10

sample = [86.0, 133.0, 75.0, 22.0, 11.0, 144.0, 78.0, 122.0, 8.0, 146.0, 33.0, 41.0, 99.0]
mean = 50.0
lambd = 1/50


d = lib.estadistico_smirnov(sample, lambda x : expon.cdf(x, scale=50))

m = 10_000
pvalor = 0
for _ in range(m):
    pvalor += int(lib.estadistico_smirnov_sim(len(sample)) > d) / m

print("d: ", d)
print("p-valor: ", pvalor)

# --- Codigo para checkear que da lo mismo usar uniformes -----

# def estadistico_smirnov_sim_exp(n):
#     sample = sorted([random.expovariate(lambd=lambd) for _ in range(n)])

#     d = lib.estadistico_smirnov(sample, lambda x : expon.cdf(x, scale=50))

#     return d

# m = 10_000
# pvalor = 0
# for _ in range(m):
#     pvalor += int(estadistico_smirnov_sim_exp(len(sample)) > d) / m
# print("p-valor exp: ", pvalor)



d:  0.3922544552361856
p-valor:  0.02259999999999992


In [70]:
# 11
import numpy as np
from scipy.stats import chi2, expon
import lib
import importlib
import math
import random
importlib.reload(lib)

"""
Binomial con n = 8, p no especificada
"""

sample = [6, 7, 3, 4, 7, 3, 7, 2, 6, 3, 7, 8, 2, 1, 3, 5, 8, 7]
values = list(range(9))
Ns = [sample.count(val) for val in values]
k = 9 # n = 8, hay 9 valores posibles

# np ~ promedio
# sabemos n = 8
# estimamos p
p = sum(sample) / (len(sample)*8)

ps = [math.comb(8, x) * (p**x) * (1 - p)**(8-x) for x in values]

T = lib.estadistico_prueba(ps, Ns)

print("T: ", T)

print("p-valor: ", chi2.sf(T, k - 1 - 1)) # usamos k - 1 - m, donde m es la cantidad de parametros que estimamos

T:  31.499330934155324
p-valor:  5.027994320424078e-05


# Ejercicio 12
Un escribano debe validar un juego en cierto programa de televisión. El mismo consiste
en hacer girar una rueda y obtener un premio según el sector de la rueda que coincida con una aguja.
Hay 10 premios posibles, y las áreas de la rueda para los distintos premios, numerados del 1 al 10, son
respectivamente:

$$31\%, 22\%, 12\%, 10\%, 8\%, 6\%, 4\%, 4\%, 2\%, 1\%.$$

Los premios con número alto (e.j. un auto 0Km) son mejores que los premios con número bajo (e.j. 2x1
para entradas en el cine). El escribano hace girar la rueda hasta que se cansa, y anota cuántas veces sale
cada sector. Los resultados, para los premios del 1 al 10, respectivamente, son:

$$ 188, 138, 87, 65, 48, 32, 30, 34, 13, 2 $$

(a) Construya una tabla con los datos disponibles

(b) Diseñe una prueba de hipótesis para determinar si la rueda es justa

(c) Defina el p-valor a partir de la hipótesis nula

(d) Calcule el p-valor bajo la hipótesis de que la rueda es justa, usando la aproximación chi cuadrado

(e) Calcule el p-valor bajo la hipótesis de que la rueda es justa, usando una simulación

In [12]:
# 12
import numpy as np
from scipy.stats import chi2, expon
import lib
import importlib
import math
import random
importlib.reload(lib)
from tabulate import tabulate

Ns = [188, 138, 87, 65, 48, 32, 30, 34, 13, 2]

ps = [.31, .22, .12, .10, .08, .06, .04, .04, .02, .01]

tablita = {
    "values": list(range(1, 11)),
    "prob observada": [v/sum(Ns) for v in Ns],
    "prob teorica": ps
}

print(tabulate(tablita, headers="keys"))

# Prueba de Pearson

"""
Bajo la hipótesis nula, la variable "en qué area cae la flecha" distribuye como en la lista ps

el estadistico de prueba entonces distribuye Chi Cuadrado
"""

T = lib.estadistico_prueba(ps, Ns)

pvalor = chi2.sf(T, 9)

print("T: ", T)
print("p-valor: ", pvalor)

m = 1_000

pvalsim = 0
for _ in range(m):
    pvalsim += int(lib.estadistico_prueba_sim(ps, sum(Ns)) > T) / m 

print("p-valor simulado: ", pvalsim)






  values    prob observada    prob teorica
--------  ----------------  --------------
       1        0.295133              0.31
       2        0.216641              0.22
       3        0.136578              0.12
       4        0.102041              0.1
       5        0.0753532             0.08
       6        0.0502355             0.06
       7        0.0470958             0.04
       8        0.0533752             0.04
       9        0.0204082             0.02
      10        0.00313972            0.01
T:  9.810370888711903
p-valor:  0.3660538998868262
p-valor simulado:  0.35400000000000026


In [65]:
import numpy as np
from scipy.stats import chi2, expon
import lib
import importlib
import math
import random
importlib.reload(lib)

# 13

sample = [random.expovariate(lambd=1) for _ in range(30)]

d = lib.estadistico_smirnov(sample, lambda x : expon.cdf(x))

pval = 0

m = 10_000

for _ in range(m):
    pval += int(lib.estadistico_smirnov_sim(30) < d) / m

print(d)


print(pval)


0.12663715497966163
0.32669999999998034


In [82]:
# 14
import numpy as np
from scipy.stats import t, norm
import lib
import importlib
import math
import random
from typing import List
importlib.reload(lib)

n = 10
sample: List[float] = [t.rvs(df = 11) for _ in range(n)]
d = lib.estadistico_smirnov(sample, lambda x : norm.cdf(x))
print(d)

pval = 0

m = 10_000

for _ in range(m):
    pval += int(lib.estadistico_smirnov_sim(n) > d) / m

print(pval)

0.22981018342040072
0.5869999999999517


In [108]:
import numpy as np
from scipy.stats import expon
import lib
import importlib
import math
import random
from typing import List
importlib.reload(lib)

# 15


sample = [1.6, 10.3, 3.5, 13.5, 18.4, 7.7, 24.3, 10.7, 8.4, 4.9, 7.9, 12, 16.2, 6.8, 14.7]

mean = sum(sample)/len(sample)

d = lib.estadistico_smirnov(sample, lambda x : expon.cdf(x, scale=mean))

pvalor_u = 0
pvalor_exp = 0

m = 10_000

for _ in range(m):
    pvalor_u += int(lib.estadistico_smirnov_sim(len(sample)) < d)/ m

    sample_exp = [random.expovariate(lambd = 1/mean) for _ in range(len(sample))]

    sample_exp_mean = sum(sample_exp)/len(sample_exp)

    pvalor_exp += int(lib.estadistico_smirnov(sample_exp, lambda x : expon.cdf(x, scale=sample_exp_mean)) < d)/ m

print("d: ", d)
print("pvalor: ", pvalor_u)
print("pvalor (estimacion de 'mayor' precision): ", pvalor_exp)

d:  0.26949936321059237
pvalor:  0.8104999999999271
pvalor (estimacion de 'mayor' precision):  0.9479999999999119


In [146]:
# 16
import numpy as np
from scipy.stats import norm
import lib
import importlib
import math
import random
from typing import List
importlib.reload(lib)

sample = [91.9, 97.8, 111.4, 122.3, 105.4, 95.0, 103.8, 99.6, 96.6, 119.3, 104.8, 101.7]

n = len(sample)

esp  = sum(sample)/n
esp2 = sum(x**2 for x in sample)/n
var = esp2 - esp**2

stddev = math.sqrt(var)


print("mu estimado: ",  esp, "sigma estimado: ", stddev)

d = lib.estadistico_smirnov(sample, lambda x : norm.cdf(x, loc=esp, scale=stddev))

print(d)

m = 10_000
pval = 0
pval_2 = 0

for _ in range(m):
    pval += int(lib.estadistico_smirnov_sim(n) > d) / m

    sample_2 = np.random.normal(esp, stddev, size=n)

    esp_2  = sum(sample_2)/n
    esp2_2 = sum(x**2 for x in sample_2)/n
    var_2 = esp2_2 - (esp_2**2)
    stddev_2 = math.sqrt(var_2)

    pval_2 += int(lib.estadistico_smirnov(sample_2, lambda x : norm.cdf(x, loc=esp_2, scale=stddev_2)) > d) / m




print("p-valor: ", pval)
print("p-valor mas certero: ", pval_2)

mu estimado:  104.13333333333334 sigma estimado:  8.997345287484546
0.1940209762022943
p-valor:  0.6865999999999407
p-valor mas certero:  0.2536999999999884


In [183]:
import numpy as np
from scipy.stats import poisson, chi2
import lib
import importlib
import math
import random
from typing import List
importlib.reload(lib)

sample = [0, 2, 5, 2, 2, 6, 3, 1, 2, 2, 4, 5, 4, 5, 1, 3, 3, 6, 4, 1]


values = list(range(0, 7))
Ns = [sample.count(val) for val in values]
k = 7
ps = [poisson.pmf(val, 3) for val in values[0:-1]]
ps += [1 - sum(ps)]

T = lib.estadistico_prueba(ps, Ns)
print(T)

pval = chi2.sf(T, df=k-1)
print("primer pval: ", pval)

m = 10_000
pval2 = 0

for _ in range(m):
    pval2 += int(lib.estadistico_prueba_sim_binomiales(ps, sum(Ns)) > T) / m

print("super pval: ", pval2)


1.1297816897573754
primer pval:  0.9802109144943139
super pval:  0.9777999999999086
